In [17]:
import requests
import json
%pip install pymongo
from pymongo import MongoClient
import pandas as pd


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [43]:
# First we want to grab all relevant businesses that match the query request

url = "https://yelp-reviews.p.rapidapi.com/business-search"

business_query = "Gyms"


querystring = {"query":business_query,"location":"Vancouver, BC, Canada","start":"0","yelp_domain":"yelp.com"}

headers = {
	"X-RapidAPI-Key": "52f0c4fb7cmsh68305c1877afa13p1710b0jsn6ea7e5079542",
	"X-RapidAPI-Host": "yelp-reviews.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())



{'status': 'OK', 'request_id': 'dba028a6-9bf1-47b3-baae-1c1ab31efc41', 'data': [{'id': 'W130LnKQUbMtkeWZlITtjA', 'alias': 'robert-lee-ymca-vancouver', 'name': 'Robert Lee YMCA', 'address': '955 Burrard Street, Vancouver, BC V6Z 1Y2, Canada', 'latitude': 49.28191794448205, 'longitude': -123.12544287350603, 'business_page_link': 'https://www.yelp.com/biz/robert-lee-ymca-vancouver', 'rating': 3.8, 'review_count': 99, 'price_range': '', 'photo': 'https://s3-media0.fl.yelpcdn.com/bphoto/U3A3yInGs4IWLQ6OuGL-sg/348s.jpg', 'photos_page_link': 'https://www.yelp.com/biz_photos/W130LnKQUbMtkeWZlITtjA', 'phone': '+16046899622', 'country': 'CA', 'neighborhoods': ['West End']}, {'id': 'tmJkLee4vC9XT5oo3jI1Sg', 'alias': 'equinox-w-georgia-st-vancouver-4', 'name': 'Equinox W Georgia St', 'address': '1131 West Georgia Street, Vancouver, BC V6E 4G2, Canada', 'latitude': 49.2862122481809, 'longitude': -123.123522053777, 'business_page_link': 'https://www.yelp.com/biz/equinox-w-georgia-st-vancouver-4', 'r

In [44]:
import os
import json
import pprint
# Create the DATA folder if it doesn't exist
if not os.path.exists("DATA"):
    os.makedirs("DATA")

# Specify the file path
file_path = "DATA/response_data.json"

# Save the dictionary to the JSON file
with open(file_path, 'w') as f:
    json.dump(response.json(), f)

# Load the JSON data from the file
with open(file_path, 'r') as f:
    data = json.load(f)

# Pretty print the JSON data
pprint.pprint(data)


Businesses = data
BusinessIds = pd.DataFrame(columns = ["businessId", "link", "name", "rating"])

for business in Businesses['data']:
    BusinessIds.loc[len(BusinessIds)] = [business['id'], business['business_page_link'], business['name'], business['rating']]

BusinessIds



{'data': [{'address': '955 Burrard Street, Vancouver, BC V6Z 1Y2, Canada',
           'alias': 'robert-lee-ymca-vancouver',
           'business_page_link': 'https://www.yelp.com/biz/robert-lee-ymca-vancouver',
           'country': 'CA',
           'id': 'W130LnKQUbMtkeWZlITtjA',
           'latitude': 49.28191794448205,
           'longitude': -123.12544287350603,
           'name': 'Robert Lee YMCA',
           'neighborhoods': ['West End'],
           'phone': '+16046899622',
           'photo': 'https://s3-media0.fl.yelpcdn.com/bphoto/U3A3yInGs4IWLQ6OuGL-sg/348s.jpg',
           'photos_page_link': 'https://www.yelp.com/biz_photos/W130LnKQUbMtkeWZlITtjA',
           'price_range': '',
           'rating': 3.8,
           'review_count': 99},
          {'address': '1131 West Georgia Street, Vancouver, BC V6E 4G2, Canada',
           'alias': 'equinox-w-georgia-st-vancouver-4',
           'business_page_link': 'https://www.yelp.com/biz/equinox-w-georgia-st-vancouver-4',
           '

In [96]:
Results_final = pd.DataFrame(columns = ["gym_name", "overall_rating", "review_text", "date"])

pages = 5


for index, row in BusinessIds.iterrows():
    url = "https://yelp-reviews.p.rapidapi.com/business-reviews"

    
    business_id = row['businessId']
    business_name = row['name']
    business_rating = row['rating']
    

    BUSINESS_ID = business_id
    

    querystring = {"business_id":BUSINESS_ID,"page":"1","page_size":"10","num_pages":"5","language":"en"}

    headers = {
        "X-RapidAPI-Key": "52f0c4fb7cmsh68305c1877afa13p1710b0jsn6ea7e5079542",
        "X-RapidAPI-Host": "yelp-reviews.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    reviews_data = response.json()

    reviews_data = reviews_data['data']['reviews']

    for review in reviews_data:
        text = review['review_text']
        date = review['review_datetime_utc']

        Results_final.loc[len(Results_final)] = [business_name, business_rating, text, date]


Results_final









,gym_name,overall_rating,review_text,date
0,Robert Lee YMCA,3.8,I used to work here and absolutely loved my ti...,2023-11-08T00:12:30Z
1,Robert Lee YMCA,3.8,Facility is too small for the number of people...,2024-01-24T02:49:13Z
2,Robert Lee YMCA,3.8,It was a super friendly gym yet very crowded a...,2023-11-21T10:58:15Z
3,Robert Lee YMCA,3.8,Always amazing. Using the facility for many y...,2023-08-14T00:15:11Z
4,Robert Lee YMCA,3.8,Just joined and really impressed with the clea...,2023-04-24T02:26:31Z
...,...,...,...,...
231,Dynasty Gym,4.4,Came in saying I didn't know that much about w...,2017-06-23T02:01:51Z
232,Dynasty Gym,4.4,This is my favorite hangout in the city and th...,2019-07-15T01:16:44Z
233,Dynasty Gym,4.4,I have been going to Dynasty for over 2 years ...,2016-09-29T21:17:18Z
234,Dynasty Gym,4.4,"3.5/5\n\nDynasty is a GREAT weightlifting gym,...",2017-03-05T22:00:11Z


In [97]:
Results_final.to_csv("final_results.csv")